In [2]:
import cv2
import mediapipe as mp
import numpy as np
import asyncio
import platform
import nest_asyncio
import sys

# Apply nest_asyncio to allow nested event loops in Jupyter-like environments
nest_asyncio.apply()

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Tic-Tac-Toe game state
board = [[' ' for _ in range(3)] for _ in range(3)]
current_player = 'X'
game_over = False
winner = None

# Window and grid settings
width, height = 640, 480
cell_size = 150
grid_offset_x, grid_offset_y = 50, 50

def draw_board(frame):
    """Draw the Tic-Tac-Toe board and pieces on the frame."""
    frame = cv2.rectangle(frame, (0, 0, width, height), (255, 255, 255), -1)  # White background
    for i in range(1, 3):
        # Draw grid lines
        cv2.line(frame, (grid_offset_x + i * cell_size, grid_offset_y), 
                 (grid_offset_x + i * cell_size, grid_offset_y + 3 * cell_size), (0, 0, 0), 2)
        cv2.line(frame, (grid_offset_x, grid_offset_y + i * cell_size), 
                 (grid_offset_x + 3 * cell_size, grid_offset_y + i * cell_size), (0, 0, 0), 2)
    # Draw X and O
    for i in range(3):
        for j in range(3):
            if board[i][j] != ' ':
                cv2.putText(frame, board[i][j], 
                           (grid_offset_x + j * cell_size + 50, grid_offset_y + i * cell_size + 100),
                           cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255) if board[i][j] == 'X' else (255, 0, 0), 3)

def check_winner():
    """Check if there's a winner or a draw."""
    global winner, game_over
    # Check rows, columns, diagonals
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != ' ':
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != ' ':
            return board[0][i]
    if board[0][0] == board[1][1] == board[2][2] != ' ':
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != ' ':
        return board[0][2]
    # Check for draw
    if all(board[i][j] != ' ' for i in range(3) for j in range(3)):
        return 'Draw'
    return None

def is_hand_closed(landmarks):
    """Check if the hand is closed (all fingers down except thumb)."""
    finger_tips = [4, 8, 12, 16, 20]  # Thumb, index, middle, ring, pinky
    finger_mcp = [2, 5, 9, 13, 17]   # Metacarpophalangeal joints
    for tip, mcp in zip(finger_tips[1:], finger_mcp[1:]):  # Exclude thumb
        if landmarks[tip].y < landmarks[mcp].y:  # Finger tip above MCP means finger is up
            return False
    return True

def get_selected_cell(x, y):
    """Map hand position to grid cell."""
    if (grid_offset_x <= x <= grid_offset_x + 3 * cell_size and 
        grid_offset_y <= y <= grid_offset_y + 3 * cell_size):
        row = int((y - grid_offset_y) // cell_size)
        col = int((x - grid_offset_x) // cell_size)
        if 0 <= row < 3 and 0 <= col < 3:
            return row, col
    return None

async def main():
    global current_player, game_over, winner
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while not game_over:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)  # Mirror the frame
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        draw_board(frame)
        selected_cell = None
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                # Get index finger tip coordinates
                x = int(hand_landmarks.landmark[8].x * width)
                y = int(hand_landmarks.landmark[8].y * height)
                selected_cell = get_selected_cell(x, y)
                if selected_cell:
                    row, col = selected_cell
                    # Highlight selected cell
                    cv2.rectangle(frame, 
                                 (grid_offset_x + col * cell_size, grid_offset_y + row * cell_size),
                                 (grid_offset_x + (col + 1) * cell_size, grid_offset_y + (row + 1) * cell_size),
                                 (0, 255, 0), 2)
                    # Check for confirm gesture (closed hand)
                    if is_hand_closed(hand_landmarks.landmark):
                        if board[row][col] == ' ':
                            board[row][col] = current_player
                            winner = check_winner()
                            if winner:
                                game_over = True
                            current_player = 'O' if current_player == 'X' else 'X'

        # Display game status
        if game_over:
            status = f"Winner: {winner}" if winner != 'Draw' else "Draw"
            cv2.putText(frame, status, (grid_offset_x, grid_offset_y + 3 * cell_size + 50),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        else:
            cv2.putText(frame, f"Player {current_player}'s turn", 
                       (grid_offset_x, grid_offset_y + 3 * cell_size + 50),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        cv2.imshow('Tic-Tac-Toe', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        await asyncio.sleep(0.03)  # ~30 FPS

    cap.release()
    cv2.destroyAllWindows()

# Run the main function appropriately based on the environment
if platform.system() == "Emscripten":
    asyncio.ensure_future(main())
elif 'ipykernel' in sys.modules:
    # In Jupyter, use the existing event loop
    await main()
else:
    if __name__ == "__main__":
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.create_task(main())
        else:
            loop.run_until_complete(main())